In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
% matplotlib inline

# Vectors and Models
We've more or less settled on using TFIDF as the vectorizer of record. This is because we are looking at frequencies across a range of documents. We went to all the trouble of creating corpora of various lengths in order to have a better experimental field for the research.

Edit of 4 Apr 2017: I'm still not sure that my method is the best method possible. I'd like to try doing the same thing with pipelines to perhaps expedite the process, or at least make it grid search ready.

Twain = 1 
Wilde = 2 
Lincoln = 3
///
D_Twain = 10 
D_Wilde = 20 
D_Lincoln = 30
///
Modern = 100

1000-, 500-, and 100-record samples for each original writer - Twain, Wilde, Lincoln, and Modern

In [4]:
df_1k_H = pd.read_csv('_CSVs/df_1k_H.csv', index_col=0)
df_1k_W = pd.read_csv('_CSVs/df_1k_W.csv', index_col=0)
df_1k_S = pd.read_csv('_CSVs/df_1k_S.csv', index_col=0)

df_500_H = pd.read_csv('_CSVs/df_500_H.csv', index_col=0)
df_500_W = pd.read_csv('_CSVs/df_500_W.csv', index_col=0)
df_500_S = pd.read_csv('_CSVs/df_500_S.csv', index_col=0)

df_100_H = pd.read_csv('_CSVs/df_100_H.csv', index_col=0)
df_100_W = pd.read_csv('_CSVs/df_100_W.csv', index_col=0)
df_100_S = pd.read_csv('_CSVs/df_100_S.csv', index_col=0)

## Pipelines
This is a bit odd. It seems to me that since we're ultimately using the TF-IDF transformer, we need multiple documents to compare frequencies of. So, to simulate 'multiple documents' when I had already combined everything into one document, I split the several authors into groups of different numbers of observations (1000, 500, and 100 rows) at different levels (cHaracter, Word, and Sentence).

In [35]:
#wild_data = pd.read_csv('quotes.csv')

In [36]:
X_1k_H = df_1k_H['0'].values #1000 observations at the character level
y_1k_H = df_1k_H['code'].values

In [37]:
X_1k_W = df_1k_W['0'].values #1000 observations at the word level
y_1k_W = df_1k_W['code'].values

In [39]:
X_1k_S = df_1k_S['0'].values #1000 observations at the sentence level
y_1k_S = df_1k_S['code'].values

In [44]:
pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB())
    ])

In [52]:
parameters = {
    #'vect__decode_error': ('ignore'),
    'vect__analyzer': ('word', 'char'),
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (2, 2), (3, 3), (4, 4)),  # individually checking uni- through tetragrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__fit_prior': (True, False),
    #'clf__n_iter': (10, 50, 80),
}

In [60]:
grid_search = GridSearchCV(pipeline, parameters, scoring='precision_macro', n_jobs=-1, verbose=1)

In [61]:
grid_search.fit(X_1k_H, y_1k_H)

Fitting 3 folds for each of 120 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 53.4min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 101.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__analyzer': ('word', 'char'), 'vect__ngram_range': ((1, 1), (1, 2), (2, 2), (3, 3), (4, 4)), 'clf__alpha': (1e-05, 1e-06), 'clf__fit_prior': (True, False), 'vect__max_df': (0.5, 0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='precision_macro', verbose=1)

In [62]:
grid_search.best_params_

{'clf__alpha': 1e-05,
 'clf__fit_prior': True,
 'vect__analyzer': 'char',
 'vect__max_df': 1.0,
 'vect__ngram_range': (3, 3)}

In [63]:
grid_search.best_score_

0.93701123599435088